In [8]:
import pandas as pd
import numpy as np

mimic = "../mimic-preprocessed/"
snomedlocation = "../snomed+icd_vocab/"

In [9]:
from neo4j import GraphDatabase

class __Neo4jConnection:
    
    # db = ""
    
    def __init__(self, uri, user, pwd, db):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        self.db = db
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, return_response = True):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=self.db) if self.db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        if return_response:
            return response

def create_neo4j_connection(neo_usr, neo_pwd, neo_host, neo_bolt, neo_db ):

    n4j_bolt_url = f"bolt://{neo_host}:{neo_bolt}"

    n4j_conn = __Neo4jConnection(uri=n4j_bolt_url, 
                            user=neo_usr,              
                            pwd=neo_pwd, 
                            db = neo_db)

    return n4j_conn

n4j = create_neo4j_connection(neo_usr = "neo4j", neo_pwd = "passw0rd", 
    neo_host = "localhost", 
    neo_db = "neo4j", 
    neo_bolt  = 7687)

n4j.query("CALL gds.graph.list() YIELD graphName")

Query failed: There is no procedure with the name `gds.graph.list` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.


# Load mimic

In [6]:
mimic_df = pd.read_csv(mimic + "mimic2neo_dxs.csv")
mimic_df

,row_id,subject_id,hadm_id,seq_num,icd9_code,icd9_major
0,1297,109,172335,1.0,40301,403
1,1298,109,172335,2.0,486,486
2,1299,109,172335,3.0,58281,582
3,1300,109,172335,4.0,5855,585
4,1301,109,172335,5.0,4254,425
...,...,...,...,...,...,...
603477,639798,97503,188195,2.0,20280,202
603478,639799,97503,188195,3.0,V5869,V58
603479,639800,97503,188195,4.0,V1279,V12
603480,639801,97503,188195,5.0,5275,527


# Explore SNOMED CT files

In [4]:
!ls /snomed+icd_vocab

CONCEPT.csv	      CONCEPT_RELATIONSHIP.csv	DRUG_STRENGTH.csv
CONCEPT_ANCESTOR.csv  CONCEPT_SYNONYM.csv	RELATIONSHIP.csv
CONCEPT_CLASS.csv     DOMAIN.csv		VOCABULARY.csv


In [10]:
concept = pd.read_csv(snomedlocation + "CONCEPT.csv", sep="\t", low_memory=False)
concept_rel = pd.read_csv(snomedlocation + "CONCEPT_RELATIONSHIP.csv", sep="\t", low_memory=False)
# drug_stength = pd.read_csv(snomedlocation + "DRUG_STRENGTH.csv", sep="\t", low_memory=False) # empty table
# concept_ance = pd.read_csv(snomedlocation + "CONCEPT_ANCESTOR.csv", sep="\t", low_memory=False) # not needed, we just map directly from relationship table
# concept_syn = pd.read_csv(snomedlocation + "CONCEPT_SYNONYM.csv", sep="\t", low_memory=False) # i think we don't need this here
relationship = pd.read_csv(snomedlocation + "RELATIONSHIP.csv", sep="\t", low_memory=False) # mapping to human readable naming and reverse relationship
concept_class = pd.read_csv(snomedlocation + "CONCEPT_CLASS.csv", sep="\t", low_memory=False)
# domain = pd.read_csv(snomedlocation + "DOMAIN.csv", sep="\t", low_memory=False)
# vocablulary = pd.read_csv(snomedlocation + "VOCABULARY.csv", sep="\t", low_memory=False)

# Concepts

In [25]:
concept.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN


In [12]:
pd.set_option("display.max_columns", None)
concept.groupby(['vocabulary_id', 'domain_id']).size().unstack(fill_value=0)

domain_id,Condition,Condition Status,Cost,Device,Drug,Episode,Gender,Geography,Meas Value,Meas Value Operator,Measurement,Metadata,Observation,Payer,Plan,Plan Stop Reason,Procedure,Provider,Race,Relationship,Revenue Code,Route,Spec Anatomic Site,Spec Disease Status,Specimen,Sponsor,Type Concept,Unit,Visit
vocabulary_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CDM,0,0,0,0,0,0,0,0,0,0,0,1045,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Concept Class,0,0,0,0,0,0,0,0,0,0,0,415,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Condition Status,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Condition Type,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,118,0,0
Cost,0,0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cost Type,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0
Death Type,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0
Device Type,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0
Domain,0,0,0,0,0,0,0,0,0,0,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## ICD 9 Part

In [13]:
concept[concept.vocabulary_id == "ICD9CM"].head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1775,44830884,Ill-defined intestinal infections,Condition,ICD9CM,3-dig nonbill code,NaN,009,19700101,20991231,NaN
1776,44829703,"Tuberculosis of lung, infiltrative, bacteriolo...",Condition,ICD9CM,5-dig billing code,NaN,011.02,19700101,20991231,NaN
1777,44830892,"Tuberculous bronchiectasis, tubercle bacilli n...",Condition,ICD9CM,5-dig billing code,NaN,011.54,19700101,20991231,NaN
1778,44819320,Tuberculoma of meninges,Condition,ICD9CM,4-dig nonbill code,NaN,013.1,19700101,20991231,NaN
1779,44819327,Tuberculosis of bones and joints,Condition,ICD9CM,3-dig nonbill code,NaN,015,19700101,20991231,NaN


In [76]:
# sanity check that all mimic codes are in the snomed one:
test_icd = concept[concept.vocabulary_id == "ICD9CM"]
# test_icd = test_icd[test_icd.domain_id.isin(["Condition", "Observation"])]
test_icd["concept_code_processed"] = test_icd.apply(lambda r: r.concept_code.replace(".", ""), axis = 1)
mimic_df[~(mimic_df.icd9_code.isin(test_icd.concept_code_processed))]

/var/folders/ws/4g3jm2613sdd45_fr0gktcyjl4jwf6/T/ipykernel_2856/2508943364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_icd["concept_code_processed"] = test_icd.apply(lambda r: r.concept_code.replace(".", ""), axis = 1)


,row_id,subject_id,hadm_id,seq_num,icd9_code,icd9_major


In [80]:
mimic_icd = mimic_df.merge(test_icd, how = "left", 
                           left_on = "icd9_code",
                           right_on = "concept_code_processed")
mimic_icd.groupby(['domain_id'])['domain_id'].count()

domain_id
Condition      547089
Measurement       760
Observation     53267
Procedure        2366
Name: domain_id, dtype: int64

In [82]:
concept[concept.vocabulary_id == "ICD9CM"].groupby(['domain_id'])['domain_id'].count()

domain_id
Condition      15536
Measurement      149
Observation     1701
Procedure        178
Name: domain_id, dtype: int64

In [104]:
sm_id_map = concept_rel[(concept_rel.concept_id_1.isin(test_icd.concept_id)) & (concept_rel.concept_id_2.isin(concept_snomed.concept_id))]

sm_id_map.groupby(['relationship_id'])['relationship_id'].count()

relationship_id
Maps to          20576
Maps to value      398
Name: relationship_id, dtype: int64

In [95]:
sm_id_map[sm_id_map.relationship_id == "Maps to value"]

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
5641375,44828618,4216752,Maps to value,20141001,20991231,NaN
5641376,44834226,4274025,Maps to value,20141001,20991231,NaN
5641377,44824953,440940,Maps to value,20141001,20991231,NaN
5641435,44836599,432262,Maps to value,20220826,20991231,NaN
5641436,44821038,442088,Maps to value,20220826,20991231,NaN
...,...,...,...,...,...,...
9981277,44827296,4291005,Maps to value,20141001,20991231,NaN
9981320,44833096,4083382,Maps to value,20210220,20991231,NaN
9981321,44820428,4012185,Maps to value,20210220,20991231,NaN
9981322,44820428,4229604,Maps to value,20210220,20991231,NaN


In [97]:
concept[concept.concept_id == 4216752]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
970586,4216752,Drug therapy,Procedure,SNOMED,Procedure,S,416608005,20050731,20991231,NaN


In [105]:
concept_snomed[concept_snomed.concept_id.isin(sm_id_map.concept_id_2)].groupby(['domain_id'])['domain_id'].count()

domain_id
Condition              9822
Meas Value                1
Measurement             107
Observation             787
Procedure               209
Spec Disease Status       1
Name: domain_id, dtype: int64

## SNOMED Part

In [98]:
concept[concept.vocabulary_id == "SNOMED"].head()
concept_snomed = concept[concept.vocabulary_id.isin(["SNOMED"])]
concept_snomed.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
228217,4000935,BITTER-3,Drug,SNOMED,Pharma/Biol Product,NaN,100000000,20020131,20090731,D
228218,3627602,SIGVARIS 200 medium/plus/17348 class 2(23-32mm...,Device,SNOMED,Physical Object,NaN,10000001000001107,19700101,20090401,D
228219,4001098,Radiating chest pain,Condition,SNOMED,Clinical Finding,S,10000006,20020131,20991231,NaN
228220,37392117,Urine tryptophan:creatinine ratio,Measurement,SNOMED,Observable Entity,S,1000001000000103,20160401,20991231,NaN
228221,3587688,Gelcosal 5% gel 50g,Observation,SNOMED,Context-dependent,NaN,1000001000001104,19700101,20040131,D


In [101]:
concept_snomed.groupby(['concept_class_id', 'domain_id']).size().unstack(fill_value=0)

domain_id,Condition,Device,Drug,Gender,Meas Value,Meas Value Operator,Measurement,Metadata,Observation,Procedure,Provider,Race,Relationship,Route,Spec Anatomic Site,Spec Disease Status,Specimen,Type Concept,Unit,Visit
concept_class_id,,,,,,,,,,,,,,,,,,,,
Admin Concept,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1118,0,0
Attribute,0,0,0,0,0,0,0,4,1320,0,0,0,0,0,0,0,0,0,0,0
Biological Function,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
Body Structure,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38418,0,0,0,0,0
Clinical Drug,0,1,7775,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0
Clinical Drug Form,0,0,7004,0,0,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0
Clinical Finding,178817,1,0,10,0,0,100,11,17493,0,0,151,0,0,0,0,0,0,0,0
Context-dependent,71,1452,3447,0,0,0,2,13,76074,34,0,0,0,6,0,0,0,0,0,0
Disposition,0,0,0,0,0,0,0,0,392,0,0,0,0,0,0,0,0,0,0,0


In [103]:
concept_snomed[(concept_snomed.concept_class_id == "Clinical Finding") & (concept_snomed.domain_id == "Measurement")]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
230086,604783,Human immunodeficiency virus Centers for Disea...,Measurement,SNOMED,Clinical Finding,S,1003681006,20210131,20991231,NaN
230091,600513,Human immunodeficiency virus Centers for Disea...,Measurement,SNOMED,Clinical Finding,S,1003685002,20210131,20991231,NaN
233910,765180,Influenza A virus subtype H5 asian strain dete...,Measurement,SNOMED,Clinical Finding,S,1011000124101,20110801,20991231,NaN
243901,4013735,HIV CDC category A1,Measurement,SNOMED,Clinical Finding,NaN,103406000,20020131,20210131,U
243905,4013736,HIV CDC category A2,Measurement,SNOMED,Clinical Finding,NaN,103407009,20020131,20210131,D
...,...,...,...,...,...,...,...,...,...,...
1229389,764933,Virus not detected,Measurement,SNOMED,Clinical Finding,S,871000124102,20110801,20991231,NaN
1234067,618956,Sustained viral response,Measurement,SNOMED,Clinical Finding,S,879968001,20210131,20991231,NaN
1234641,764938,Influenza A virus not detected,Measurement,SNOMED,Clinical Finding,S,881000124104,20110801,20991231,NaN
1239272,764942,Influenza A virus subtype H1 not detected,Measurement,SNOMED,Clinical Finding,S,891000124101,20110801,20991231,NaN


### Number of relationships per domains in SNOMED

In [54]:
concept_rel_snomed = concept_rel[(concept_rel.concept_id_1.isin(concept_snomed.concept_id)) & (concept_rel.concept_id_2.isin(concept_snomed.concept_id))]
concept_rel_snomed = concept_rel_snomed.merge(relationship, on = "relationship_id")

concept_rel_snomed.head()

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason,relationship_name,is_hierarchical,defines_ancestry,reverse_relationship_id,relationship_concept_id
0,4232951,40642538,Has status,20220128,20991231,NaN,Has status,0,0,Status of,33066
1,3623288,40642538,Has status,20220128,20991231,NaN,Has status,0,0,Status of,33066
2,46135796,40642538,Has status,20220128,20991231,NaN,Has status,0,0,Status of,33066
3,4233958,40642537,Has status,20220128,20991231,NaN,Has status,0,0,Status of,33066
4,3623289,40642538,Has status,20220128,20991231,NaN,Has status,0,0,Status of,33066


In [55]:
concept_snomed["domain_1"] = concept_snomed.domain_id
concept_snomed["domain_2"] = concept_snomed.domain_id

concept_rel_snomed = concept_rel_snomed.merge(
    how = "left",
    right = concept_snomed[["concept_id", "domain_1"]], 
    left_on = "concept_id_1", 
    right_on = "concept_id")

concept_rel_snomed = concept_rel_snomed.merge(
    how = "left",
    right = concept_snomed[["concept_id", "domain_2"]], 
    left_on = "concept_id_2", 
    right_on = "concept_id")

/var/folders/ws/4g3jm2613sdd45_fr0gktcyjl4jwf6/T/ipykernel_2856/1334873358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concept_snomed["domain_1"] = concept_snomed.domain_id
/var/folders/ws/4g3jm2613sdd45_fr0gktcyjl4jwf6/T/ipykernel_2856/1334873358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concept_snomed["domain_2"] = concept_snomed.domain_id


In [48]:
concept_rel_snomed

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason,concept_id_x,domain_1,concept_id_y,domain_2
0,4232951,40642538,Has status,20220128,20991231,NaN,4232951,Spec Anatomic Site,40642538,Metadata
1,4232951,40642539,Has Module,20220128,20991231,NaN,4232951,Spec Anatomic Site,40642539,Metadata
2,3623288,40642538,Has status,20220128,20991231,NaN,3623288,Observation,40642538,Metadata
3,3623288,35631990,Has Module,20220128,20991231,NaN,3623288,Observation,35631990,Metadata
4,46135796,40642538,Has status,20220128,20991231,NaN,46135796,Drug,40642538,Metadata
...,...,...,...,...,...,...,...,...,...,...
16069415,4233251,40642539,Has Module,20220128,20991231,NaN,4233251,Observation,40642539,Metadata
16069416,3623287,40642538,Has status,20220128,20991231,NaN,3623287,Device,40642538,Metadata
16069417,3623287,35631990,Has Module,20220128,20991231,NaN,3623287,Device,35631990,Metadata
16069418,46001337,40642538,Has status,20220128,20991231,NaN,46001337,Drug,40642538,Metadata


In [56]:
overview_df = concept_rel_snomed.groupby(['domain_1', 'domain_2']).relationship_name.value_counts().unstack(fill_value=0)
overview_df

relationship_name      ARP of (SNOMED)  Absorbability of (SNOMED)  \
domain_1  domain_2                                                  
Condition Condition                  0                          0   
          Device                     0                          0   
          Drug                       0                          0   
          Gender                     0                          0   
          Meas Value                 0                          0   
...                                ...                        ...   
Visit     Condition                  0                          0   
          Metadata                   0                          0   
          Observation                0                          0   
          Procedure                  0                          0   
          Visit                      0                          0   

relationship_name      Access device used by (SNOMED)  Access of (SNOMED)  \
domain_1  domain_2                                                          
Condition Condition                                 0                   0   
          Device                                    0                   0   
          Drug                                      0                   0   
          Gender                                    0                   0   
          Meas Value                                0                   0   
...                                               ...                 ...   
Visit     Condition                                 0                   0   
          Metadata                                  0                   0   
          Observation                               0                   0   
          Procedure                                 0                   0   
          Visit                                     0                   0   

relationship_name      Active alternative_to inactive (SNOMED)  \
domain_1  domain_2                                               
Condition Condition                                          2   
          Device                                             0   
          Drug                                               0   
          Gender                                             0   
          Meas Value                                         0   
...                                                        ...   
Visit     Condition                                          0   
          Metadata                                           0   
          Observation                                        0   
          Procedure                                          0   
          Visit                                              0   

relationship_name      Active ingredient of (SNOMED)  \
domain_1  domain_2                                     
Condition Condition                                0   
          Device                                   0   
          Drug                                     0   
          Gender                                   0   
          Meas Value                               0   
...                                              ...   
Visit     Condition                                0   
          Metadata                                 0   
          Observation                              0   
          Procedure                                0   
          Visit                                    0   

relationship_name      Active possibly_equivalent_to inactive (SNOMED)  \
domain_1  domain_2                                                       
Condition Condition                                              22813   
          Device                                                     3   
          Drug                                                       0   
          Gender                                                     0   
          Meas Value                                                

In [57]:
overview_df.to_csv("relationships_domain_nodes.csv")

## Filter concepts

In [5]:
concept_filt = concept[concept.vocabulary_id.isin(["SNOMED","ICD9CM","ICD9Proc"])]
concept_filt = concept_filt.fillna("")
concept_filt.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1775,44830884,Ill-defined intestinal infections,Condition,ICD9CM,3-dig nonbill code,,009,19700101,20991231,
1776,44829703,"Tuberculosis of lung, infiltrative, bacteriolo...",Condition,ICD9CM,5-dig billing code,,011.02,19700101,20991231,
1777,44830892,"Tuberculous bronchiectasis, tubercle bacilli n...",Condition,ICD9CM,5-dig billing code,,011.54,19700101,20991231,
1778,44819320,Tuberculoma of meninges,Condition,ICD9CM,4-dig nonbill code,,013.1,19700101,20991231,
1779,44819327,Tuberculosis of bones and joints,Condition,ICD9CM,3-dig nonbill code,,015,19700101,20991231,


# filter other data souces

Keep data where concepts are in SNOMED or ICD vocabs:

In [7]:
concept_rel_filt = concept_rel[(concept_rel.concept_id_1.isin(concept_filt.concept_id)) & (concept_rel.concept_id_2.isin(concept_filt.concept_id))]

# Write to Neo4J

In [8]:
concept_filt.domain_id = concept_filt.domain_id.apply(lambda l: l.replace(" ","_"))
concept_filt.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1775,44830884,Ill-defined intestinal infections,Condition,ICD9CM,3-dig nonbill code,,009,19700101,20991231,
1776,44829703,"Tuberculosis of lung, infiltrative, bacteriolo...",Condition,ICD9CM,5-dig billing code,,011.02,19700101,20991231,
1777,44830892,"Tuberculous bronchiectasis, tubercle bacilli n...",Condition,ICD9CM,5-dig billing code,,011.54,19700101,20991231,
1778,44819320,Tuberculoma of meninges,Condition,ICD9CM,4-dig nonbill code,,013.1,19700101,20991231,
1779,44819327,Tuberculosis of bones and joints,Condition,ICD9CM,3-dig nonbill code,,015,19700101,20991231,


In [11]:
n4j.query('CREATE CONSTRAINT concept IF NOT EXISTS ON (c:Concept) ASSERT c.concept_id IS UNIQUE')

[]

In [12]:
def add_concepts(concepts, label_col):
    for label in concepts[label_col].unique():
        print(label)
        query = f'''
            UNWIND $rows AS row
            MERGE (a:Concept:{label} {{
                                concept_id: row.concept_id, 
                                name: row.concept_name, 
                                vocabulary: row.vocabulary_id, 
                                code: row.concept_code,
                                class: row.concept_class_id}})
            RETURN count(*) as total
            '''
        n = n4j.query(query, parameters = {'rows': concepts[concepts[label_col] == label].to_dict("records")})
        print("wrote " + str(n) + " nodes of domain " + label)

add_concepts(concept_filt, label_col = "domain_id")

Condition
Observation
Procedure
Measurement
Drug
Device
Metadata
Race
Spec_Anatomic_Site
Type_Concept
Specimen
Meas_Value
Visit
Route
Unit
Provider
Relationship
Gender
Spec_Disease_Status
Meas_Value_Operator


In [9]:
concept_rel_filt.head()

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,4232951,40642538,Has status,20220128,20991231,NaN
1,4232951,40642539,Has Module,20220128,20991231,NaN
2,3623288,40642538,Has status,20220128,20991231,NaN
3,3623288,35631990,Has Module,20220128,20991231,NaN
4,46135796,40642538,Has status,20220128,20991231,NaN


In [12]:
concept_rel_filt[~(concept_rel_filt.isin(["Is a", "Subsumes"]))]

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,4232951,40642538,Has status,20220128,20991231,NaN
1,4232951,40642539,Has Module,20220128,20991231,NaN
2,3623288,40642538,Has status,20220128,20991231,NaN
3,3623288,35631990,Has Module,20220128,20991231,NaN
4,46135796,40642538,Has status,20220128,20991231,NaN
...,...,...,...,...,...,...
17036797,4233251,40642539,Has Module,20220128,20991231,NaN
17036798,3623287,40642538,Has status,20220128,20991231,NaN
17036799,3623287,35631990,Has Module,20220128,20991231,NaN
17036800,46001337,40642538,Has status,20220128,20991231,NaN


In [14]:
concept_rel_filt[concept_rel_filt == "Is a"]

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
17036797,NaN,NaN,NaN,NaN,NaN,NaN
17036798,NaN,NaN,NaN,NaN,NaN,NaN
17036799,NaN,NaN,NaN,NaN,NaN,NaN
17036800,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def add_concept_relationships(rels):
#     # is a:
#     rels_isa = rels[rels.relationship_id == "Is a"]
    
#     query_isa = '''
#             UNWIND $rows AS row
#             MATCH (c1:Concept {concept_id: row.concept_id_1})
#             MATCH (c2:Concept {concept_id: row.concept_id_2})
#             MERGE (c1)-[:IS_A]->(c2)
#             RETURN count(*) as total
#             '''
#     n = n4j.query(query_isa, parameters = {'rows': rels_isa.to_dict("records")})
#     print("wrote " + str(n) + " is a rel.")
    
#     # subsumes:
#     rels_subsumes = rels[rels.relationship_id == "Subsumes"]
    
#     query_subsumes = '''
#             UNWIND $rows AS row
#             MATCH (c1:Concept {concept_id: row.concept_id_1})
#             MATCH (c2:Concept {concept_id: row.concept_id_2})
#             MERGE (c1)-[:SUBSUMES]->(c2)
#             RETURN count(*) as total
#             '''
#     n = n4j.query(query_subsumes, parameters = {'rows': rels_subsumes.to_dict("records")})
#     print("wrote " + str(n) + " subsumes rel.")
    
    # associations
    
    rels_association = rels[~(rels.relationship_id.isin(["Subsumes", "Is a"]))]
    
    query_association = '''
            UNWIND $rows AS row
            MATCH (c1:Concept {concept_id: row.concept_id_1})
            MATCH (c2:Concept {concept_id: row.concept_id_2})
            MERGE (c1)-[:ASSOCIATION{type: row.relationship_id}]->(c2)
            RETURN count(*) as total
            '''
    n = n4j.query(query_association, parameters = {'rows': rels_association.to_dict("records")})
    print("wrote " + str(n) + " other rel.")

add_concept_relationships(concept_rel_filt)

In [86]:
concept_rel_filt[concept_rel_filt.relationship_id == "Subsumes"]

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
97458,36674185,434231,Subsumes,20190131,20991231,NaN
97460,36674185,4199789,Subsumes,20190131,20991231,NaN
97463,36674185,4335897,Subsumes,20190131,20991231,NaN
97466,36674185,37309808,Subsumes,20200309,20991231,NaN
97467,36674185,37309817,Subsumes,20200309,20991231,NaN
...,...,...,...,...,...,...
16918468,35632426,35632427,Subsumes,20180731,20991231,NaN
16918475,35632428,35632429,Subsumes,20180731,20991231,NaN
16918482,35632430,35632431,Subsumes,20180731,20991231,NaN
16918490,35632432,35632433,Subsumes,20180731,20991231,NaN


# Write Mimic 

In [58]:
mimic_df

,row_id,subject_id,hadm_id,seq_num,icd9_code,icd9_major
0,1297,109,172335,1.0,40301,403
1,1298,109,172335,2.0,486,486
2,1299,109,172335,3.0,58281,582
3,1300,109,172335,4.0,5855,585
4,1301,109,172335,5.0,4254,425
...,...,...,...,...,...,...
603477,639798,97503,188195,2.0,20280,202
603478,639799,97503,188195,3.0,V5869,V58
603479,639800,97503,188195,4.0,V1279,V12
603480,639801,97503,188195,5.0,5275,527


In [59]:
n4j.query('CREATE CONSTRAINT admission IF NOT EXISTS ON (a:MIMICAdmission) ASSERT a.hadm_id IS UNIQUE')

[]

In [75]:
def add_admissions(admissions):

    query = '''
        UNWIND $rows AS row
        MERGE (a:MIMICAdmission {
                            hadm_id: row.hadm_id})
        
        WITH row, a
        MATCH (c:Concept) WHERE c.code = row.icd9_code
        MERGE (a)-[:DIAGNOSED_AS]->(c)
        
        RETURN count(*) as total
        '''
    n = n4j.query(query, parameters = {'rows': admissions.to_dict("records")})
    print("wrote " + str(n) + " admissions ")

add_admissions(mimic_df)

StopIteration: 

array([172335, 173633, 174105, ..., 161999, 189314, 168949])